In [ ]:
!pip install -U ollama openai

In [8]:
import ollama

def get_current_weather(city):
    """
    Get the current weather for a city.

    Parameters:
    city (str): The name of the city.

    Returns:
    str: The current temperature in the specified city.
    """
    return f"The current temperature in {city} is sunny"

response = ollama.chat(
    # model=""
    model="llama3.1",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant \
            that helps users with information. When users ask about\
                the weather you use the get_current_weather tool to\
                    answer.'},
        {'role': 'user', 'content': 'What is the weather in Sydney?'}],
    tools=[{
        'type': 'function',
        'name': 'get_current_weather',
        'description': 'Get the current weather for a city, the input param is city.',
        'parameters': {
            'type': 'object',
            'properties': {
                'city': {
                    'type': 'string',
                    'description': 'The name of the city',
                    'required': ['city']
                }
            }
        }
    }]
)

print(response)

{'model': 'llama3.1', 'created_at': '2024-08-08T19:37:35.601673Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': {'city': 'Sydney'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 506455291, 'load_duration': 23804625, 'prompt_eval_count': 160, 'prompt_eval_duration': 174980000, 'eval_count': 19, 'eval_duration': 303868000}


In [9]:
response

{'model': 'llama3.1',
 'created_at': '2024-08-08T19:37:35.601673Z',
 'message': {'role': 'assistant',
  'content': '',
  'tool_calls': [{'function': {'name': 'get_current_weather',
     'arguments': {'city': 'Sydney'}}}]},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 506455291,
 'load_duration': 23804625,
 'prompt_eval_count': 160,
 'prompt_eval_duration': 174980000,
 'eval_count': 19,
 'eval_duration': 303868000}

In [11]:
response['message']['tool_calls']

[{'function': {'name': 'get_current_weather',
   'arguments': {'city': 'Sydney'}}}]

In [12]:
def run_tool_calls(response):
    tool_calls = response['message']['tool_calls']
    for tool_call in tool_calls:
        function_name = tool_call['function']['name']
        arguments = tool_call['function']['arguments']
        if function_name == 'get_current_weather':
            city = arguments['city']
            temperature = get_current_weather(city)
            print(temperature)

run_tool_calls(response)

The current temperature in Sydney is sunny


# Using OpenAI API + Ollama

In [13]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

In [14]:
def check_weather(city: str) -> str:
    """Function that checks the weather."""
    return "It's sunny in " + city

In [15]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the weather in San Francisco?"}
]

In [16]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_weather",
            "description": "Get the current weather for a given city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city, e.g. San Francisco, CA",
                    },
                },
                "required": ["city"],
            },
        }
    },]



response = client.chat.completions.create(model="llama3.1", messages=messages, tools=tools)

print(response)

ChatCompletion(id='chatcmpl-33', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lyov316b', function=Function(arguments='{"city":"San Francisco"}', name='check_weather'), type='function')]))], created=1723146140, model='llama3.1', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=18, prompt_tokens=169, total_tokens=187))


In [17]:
response

ChatCompletion(id='chatcmpl-33', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lyov316b', function=Function(arguments='{"city":"San Francisco"}', name='check_weather'), type='function')]))], created=1723146140, model='llama3.1', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=18, prompt_tokens=169, total_tokens=187))

In [24]:
response.choices[0].message.tool_calls[0].function.arguments

'{"city":"San Francisco"}'